In [8]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [9]:
%pip install --upgrade PyPDF2
%pip install fitz
%pip install pymupdf
%pip install python-docx
%pip install docxcompose

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import PyPDF2
import fitz
import re
import csv

In [69]:
import fitz  # PyMuPDF
from docx import Document
from docx.shared import Pt
import re
import os

def read_definition_and_procedure_name_from_pdf(file_path, start_page=13):
    try:
        doc = fitz.open(file_path)
        text = ""
        for page_num in range(start_page - 1, doc.page_count):
            page = doc[page_num]
            text += page.get_text("text")
        return text
    except Exception as e:
        raise ValueError(f"Error reading PDF with PyMuPDF: {e}")

def extract_data_before_purposes(text):
    # Use regex to find data before "Purposes:" section
    data_and_definition_pattern = r'(.*?)(?=(?:Definition|$))'
    data_and_definition_match = re.search(data_and_definition_pattern, text, re.DOTALL | re.IGNORECASE)

    if data_and_definition_match:
        return data_and_definition_match.group(1).strip()
    else:
        return ""

def extract_purposes(text):
    # Use regex to find data before "Types of admission:" section
    purposes_pattern = r'Purposes\s*(.*?)(?=(?:Types of admission:|$))'
    purposes_match = re.search(purposes_pattern, text, re.DOTALL | re.IGNORECASE)

    if purposes_match:
        return purposes_match.group(1).strip()
    else:
        return ""

def extract_definition(text):
    # Use regex to find data before "Purposes:" section
    definition_pattern = r'Definition\s*(.*?)(?=(?:Purposes:|$))'
    definition_match = re.search(definition_pattern, text, re.DOTALL | re.IGNORECASE)

    if definition_match:
        return definition_match.group(1).strip()
    else:
        return ""

def extract_equipment(text):
    # Use regex to find data before "Procedure:" section
    equipment_pattern = r'Equipment\s*(.*?)(?=(?:Procedure:|$))'
    equipment_match = re.search(equipment_pattern, text, re.DOTALL | re.IGNORECASE)

    if equipment_match:
        return equipment_match.group(1).strip()
    else:
        return ""

def extract_procedure(text):
    # Use regex to find data after "Procedure:" heading and before the next heading
    procedure_pattern = r'Procedure\s*:(.*?)(?=(?:(?!\w+\s*:|Definition).)*\w+\s*:|$)'
    procedure_match = re.search(procedure_pattern, text, re.DOTALL | re.IGNORECASE)

    if procedure_match:
        procedure_text = procedure_match.group(1).strip()

        # Use non-greedy match to capture content before the next heading
        next_heading_pattern = r'(\w+\s*:|$)'
        next_heading_match = re.search(next_heading_pattern, procedure_text)
        
        if next_heading_match:
            procedure_text = procedure_text[:next_heading_match.start()].strip()

        return procedure_text
    else:
        return "" 
    
def write_data_to_docx(procedure_name_data, purposes_data, definition_data, equipment_data, procedure_data, output_path):
    document = Document()

    # Add a table with five columns named "Procedure Name," "Purposes," "Definition," "Equipment," and "Procedure"
    table = document.add_table(rows=2, cols=5)

    # Set column names
    column_names = ["Procedure Name", "Purposes", "Definition", "Equipment", "Procedure"]
    for i, column_name in enumerate(column_names):
        cell = table.cell(0, i)
        cell.text = column_name
        cell.paragraphs[0].runs[0].font.bold = True

    # Add data to the table
    row_cells = table.cell(1, 0)
    row_cells.text = procedure_name_data.replace('\n', ' ')

    row_cells = table.cell(1, 1)
    row_cells.text = purposes_data.replace('\n', ' ')

    row_cells = table.cell(1, 2)
    row_cells.text = definition_data.replace('\n', ' ')

    row_cells = table.cell(1, 3)
    row_cells.text = equipment_data.replace('\n', ' ')

    row_cells = table.cell(1, 4)
    row_cells.text = procedure_data.replace('\n', ' ')

    # Adjust column width and cell alignment
    for column in table.columns:
        for cell in column.cells:
            paragraphs = cell.paragraphs
            for paragraph in paragraphs:
                for run in paragraph.runs:
                    font = run.font
                    font.size = Pt(10)

    # Save the document
    document.save(output_path)

if __name__ == "__main__":
    input_file_path = '/Users/niveditareddy./Downloads/Nursing_Manual.pdf'
    output_file_path = '/Users/niveditareddy./Downloads/Nursing_StructuredData.docx'

    # Check if the file exists
    if os.path.isfile(input_file_path):
        pdf_text = read_definition_and_procedure_name_from_pdf(input_file_path, start_page=13)
        procedure_name_data = extract_data_before_purposes(pdf_text)
        purposes_data = extract_purposes(pdf_text)
        definition_data = extract_definition(pdf_text)
        equipment_data = extract_equipment(pdf_text)
        procedure_data = extract_procedure(pdf_text)

        # Write "Procedure Name," "Purposes," "Definition," "Equipment," and "Procedure" data to DOCX in a table
        write_data_to_docx(procedure_name_data, purposes_data, definition_data, equipment_data, procedure_data, output_file_path)
        
        print("Conversion completed successfully.")
    else:
        print(f"Error: The specified file '{input_file_path}' does not exist. Please check the file path.")

Conversion completed successfully.
